This notebook implements a multi-agent research and summarization system using LangGraph in a Colab environment.

Objective:

To design a workflow that processes user queries and routes them to the appropriate agent for reasoning, retrieval, or web research.

The system consists of the following workflow:

Router Agent: Determines the query intent and decides whether to invoke LLM reasoning, RAG retrieval from a dataset, or a Web Research agent.

LLM Agent: Performs reasoning on queries that require generated knowledge.

RAG Agent: Retrieves relevant information from a predefined dataset (dummy dataset included).

Web Research Agent: Fetches up-to-date information from the web when needed.

Summarization Agent: Synthesizes results from all agents into a well-structured final response, ready for PDF export.

Purpose:
This notebook demonstrates:

The construction of a multi-agent workflow using LangGraph.

Conditional routing and parallel agent execution.

Visualization of the workflow with nodes and edges.

Execution of queries at runtime with results integrated from multiple specialized agents.

It serves as a foundation for advanced multi-agent systems and research summarization workflows in Python.

In [ ]:
#CELL 1 — Install Dependencies (Colab Safe)
!pip install -q --upgrade langgraph langchain langchain-groq faiss-cpu reportlab

In [28]:
#CELL 2 — Set Groq API Key
import os

# Replace with your Groq API key
os.environ["GROQ_API_KEY"] = ""


In [29]:
#CELL 3 — Imports
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq


In [30]:
#CELL 4 — Define Shared Graph State
class AgentState(TypedDict):
    query: str
    route: Optional[str]
    llm_result: Optional[str]
    rag_result: Optional[str]
    web_result: Optional[str]
    final_answer: Optional[str]


In [31]:
#CELL 5 — Initialize Groq LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)


In [32]:
#CELL 6 — Define Router Agent
def router_agent(state: AgentState):
    query = state["query"].lower()

    if "document" in query or "internal" in query or "dataset" in query:
        route = "rag"
    elif "latest" in query or "news" in query or "web" in query:
        route = "web"
    else:
        route = "llm"

    return {"route": route}


In [33]:
#CELL 7 — Define LLM Reasoning Agent
def llm_agent(state: AgentState):
    query = state["query"]

    response = llm.invoke(f"""
You are a reasoning assistant.
Answer the following query clearly and concisely:

{query}
""")

    return {"llm_result": response.content}


In [34]:
#CELL 8 — Define RAG Agent (Dummy Knowledge Base)
# Dummy knowledge base
KNOWLEDGE_BASE = {
    "langgraph": "LangGraph is a framework for building multi-agent workflows using graph-based state machines.",
    "rag": "RAG (Retrieval-Augmented Generation) enhances LLMs by retrieving relevant documents before generation.",
    "multi agent": "Multi-agent systems consist of specialized agents collaborating to solve complex tasks."
}

def rag_agent(state: AgentState):
    query = state["query"].lower()

    retrieved_text = "No relevant internal data found."

    for key, value in KNOWLEDGE_BASE.items():
        if key in query:
            retrieved_text = value
            break

    return {"rag_result": f"[RAG Retrieval]\n{retrieved_text}"}


In [35]:
#CELL 9 — Define Web Research Agent (Simulated)def web_agent(state: AgentState):
def web_agent(state: AgentState):
    query = state["query"]

    return {
        "web_result": f"[Web Research]\nSimulated web search results for: '{query}'.\n(You can replace this with a real search API later.)"
    }


In [36]:
#CELL 10 — Define Summarization Agent (Final Node)
def summarizer_agent(state: AgentState):
    parts = []

    if state.get("llm_result"):
        parts.append("LLM Reasoning:\n" + state["llm_result"])
    if state.get("rag_result"):
        parts.append("RAG Retrieval:\n" + state["rag_result"])
    if state.get("web_result"):
        parts.append("Web Research:\n" + state["web_result"])

    final_answer = "\n\n".join(parts)

    return {"final_answer": final_answer}


In [37]:
#CELL 11 — Build the LangGraph (Your Exact Diagram)
workflow = StateGraph(AgentState)

# Nodes
workflow.add_node("start", lambda state: state)
workflow.add_node("router", router_agent)
workflow.add_node("llm", llm_agent)
workflow.add_node("rag", rag_agent)
workflow.add_node("web", web_agent)
workflow.add_node("summary", summarizer_agent)

# Edges
workflow.add_edge("start", "router")

# Router branching
workflow.add_conditional_edges(
    "router",
    lambda state: state["route"],
    {
        "llm": "llm",
        "rag": "rag",
        "web": "web",
    },
)

# Parallel paths → Summarization
workflow.add_edge("llm", "summary")
workflow.add_edge("rag", "summary")
workflow.add_edge("web", "summary")

# End
workflow.add_edge("summary", END)

# Entry point
workflow.set_entry_point("start")


In [38]:
#CELL 12 — Compile Graph
app = workflow.compile()


In [39]:
#CELL 13 — Visualize the Graph (Show Instructor)
print("\n===== LANGGRAPH STRUCTURE =====\n")
print(app.get_graph())

display(app.get_graph().draw_mermaid())



===== LANGGRAPH STRUCTURE =====

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'start': Node(id='start', name='start', data=start(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'router': Node(id='router', name='router', data=router(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'llm': Node(id='llm', name='llm', data=llm(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'rag': Node(id='rag', name='rag', data=rag(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'web': Node(id='web', name='web', data=web(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'summary': Node(id='summary', name='summary', data=summary(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', na

'---\nconfig:\n  flowchart:\n    curve: linear\n---\ngraph TD;\n\t__start__([<p>__start__</p>]):::first\n\tstart(start)\n\trouter(router)\n\tllm(llm)\n\trag(rag)\n\tweb(web)\n\tsummary(summary)\n\t__end__([<p>__end__</p>]):::last\n\t__start__ --> start;\n\tllm --> summary;\n\trag --> summary;\n\trouter -.-> llm;\n\trouter -.-> rag;\n\trouter -.-> web;\n\tstart --> router;\n\tweb --> summary;\n\tsummary --> __end__;\n\tclassDef default fill:#f2f0ff,line-height:1.2\n\tclassDef first fill-opacity:0\n\tclassDef last fill:#bfb6fc\n'

In [40]:
#CELL 14 — Runtime Input (Multiple Testing)
user_query = input("Enter your question: ")

result = app.invoke({
    "query": user_query,
    "route": None,
    "llm_result": None,
    "rag_result": None,
    "web_result": None,
    "final_answer": None,
})

print("\n========== FINAL ANSWER ==========\n")
print(result["final_answer"])


Enter your question: Explain multi-agent systems in AI

========== FINAL ANSWER ==========

LLM Reasoning:
**Multi-Agent Systems (MAS) in AI:**

A Multi-Agent System (MAS) is a type of artificial intelligence (AI) system that consists of multiple autonomous agents that interact with each other and their environment to achieve common or conflicting goals.

**Key Characteristics:**

1. **Autonomy**: Each agent operates independently, making decisions based on its own goals and knowledge.
2. **Interdependence**: Agents interact with each other and their environment to achieve their goals.
3. **Distributed Problem-Solving**: MAS can solve complex problems that are difficult or impossible for a single agent to solve.

**Components of a MAS:**

1. **Agents**: Individual entities that perceive their environment, reason, and act to achieve their goals.
2. **Environment**: The external world that agents interact with.
3. **Communication**: Agents can exchange information and coordinate their ac

In [41]:
#CELL 14 — Runtime Input (Multiple Testing)
user_query = input("Enter your question: ")

result = app.invoke({
    "query": user_query,
    "route": None,
    "llm_result": None,
    "rag_result": None,
    "web_result": None,
    "final_answer": None,
})

print("\n========== FINAL ANSWER ==========\n")
print(result["final_answer"])


Enter your question: What does the internal document say about LangGraph?

========== FINAL ANSWER ==========

RAG Retrieval:
[RAG Retrieval]
LangGraph is a framework for building multi-agent workflows using graph-based state machines.


In [42]:
#CELL 14 — Runtime Input (Multiple Testing)
user_query = input("Enter your question: ")

result = app.invoke({
    "query": user_query,
    "route": None,
    "llm_result": None,
    "rag_result": None,
    "web_result": None,
    "final_answer": None,
})

print("\n========== FINAL ANSWER ==========\n")
print(result["final_answer"])


Enter your question: What are the latest AI trends in 2025?

========== FINAL ANSWER ==========

Web Research:
[Web Research]
Simulated web search results for: 'What are the latest AI trends in 2025?'.
(You can replace this with a real search API later.)


In [ ]:
from IPython.display import display

graph = app.get_graph()

print("===== TEXT STRUCTURE =====")
print(graph)

print("\n===== VISUAL LANGGRAPH DIAGRAM =====")
display(graph.draw_mermaid())


In [ ]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    query: str
    route: Optional[str]
    llm_result: Optional[str]
    rag_result: Optional[str]
    web_result: Optional[str]
    final_answer: Optional[str]


In [43]:
# Dummy nodes (logic doesn't matter for the diagram)
def start_node(state):
    return state

def router_agent(state):
    # hard routing logic just for structure
    return {"route": "llm"}

def llm_agent(state):
    return {"llm_result": "LLM Output"}

def rag_agent(state):
    return {"rag_result": "RAG Output"}

def web_agent(state):
    return {"web_result": "Web Output"}

def summarizer_agent(state):
    return {"final_answer": "Final Answer"}


In [44]:
workflow = StateGraph(AgentState)

# Add nodes (VERTICES)
workflow.add_node("start", start_node)
workflow.add_node("router", router_agent)
workflow.add_node("llm", llm_agent)
workflow.add_node("rag", rag_agent)
workflow.add_node("web", web_agent)
workflow.add_node("summary", summarizer_agent)

# Define edges (CONNECTIONS)
workflow.add_edge("start", "router")

# Router → three parallel branches
workflow.add_conditional_edges(
    "router",
    lambda state: state["route"],
    {
        "llm": "llm",
        "rag": "rag",
        "web": "web"
    }
)

# All three → summarizer
workflow.add_edge("llm", "summary")
workflow.add_edge("rag", "summary")
workflow.add_edge("web", "summary")

# Summarizer → END
workflow.add_edge("summary", END)

# Entry point
workflow.set_entry_point("start")

app = workflow.compile()


In [45]:
from IPython.display import HTML

mermaid = app.get_graph().draw_mermaid()

HTML(f"""
<div class="mermaid">
{mermaid}
</div>
<script src="https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.min.js"></script>
<script>mermaid.initialize({{ startOnLoad: true }});</script>
""")
